In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity = np.zeros(shape=(54,2000), dtype=np.float64) #100个agent的资源量 定义成一个矩阵
sum_resource=np.zeros(shape=(1,2000), dtype=np.float64)#存放每一次迭代后所有用户资源量的总和
sum_cost=np.zeros(shape=(1,2000), dtype=np.float64)#存放每一次迭代后所有用户成本的总和
iteration_update = 1 #迭代步长 
A=np.loadtxt("Adjacency_Matrix_generator_zero.txt") #网络拓扑的邻接矩阵 对角元素要为0
D=np.zeros(shape=(54,54))# 网络的度矩阵
L=np.zeros(shape=(54,54))# 网络的拉普拉斯矩阵 L=D-A
W=np.zeros(shape=(54,54)) #可以直接采用网络对应的拉普拉斯矩阵  因为无向图的拉普拉斯矩阵天然满足行和和列和为0
iteration=[]#绘图横坐标
generator_a_values= np.loadtxt("generator_a_values.txt")#每个agent的成本函数中的超参数a
generator_b_values= np.loadtxt("generator_b_values.txt")#每个agent的成本函数中的超参数a
generator_c_values= np.loadtxt("generator_c_values.txt")#每个agent的成本函数中的超参数a
agent_gradient=np.zeros(shape=(54,2000), dtype=np.float64) 

In [3]:
#根据邻接矩阵生成网络对应的度矩阵
for i in range(0,54):
    d=0
    for j in range(0,54):
        if A[i][j]==1:
            d=d+1
    D[i][i]=d

#根据邻接矩阵和度矩阵计算拉普拉斯矩阵
L=D-A

#选择用拉普拉斯矩阵作为聚合矩阵
W=L
    
#资源量的初始化 需要给定可行的初始化 即满足平均资源约束
for i in range (0,54):
    agent_resource_quantity[i][0]=6000/54
    
#总资源量 初始化
sum=0
for i in range (0,54):
    sum=sum+agent_resource_quantity[i][0]
sum_resource[0]=sum

iteration.append(0)

#初始梯度计算
for i in range (0,54):
    agent_gradient[i][0]=2*generator_c_values[i]*agent_resource_quantity[i][0]+generator_b_values[i]

#用户总成本初始化
sum=0
for i in range(0,54):
    sum=sum+(generator_c_values[i]*(agent_resource_quantity[i][0])*(agent_resource_quantity[i][0])+generator_b_values[i]*agent_resource_quantity[i][0]+generator_a_values[i])
sum_cost[0][0]=sum

In [4]:
W

array([[ 4., -1., -1., ...,  0.,  0.,  0.],
       [-1.,  4., -1., ...,  0.,  0.,  0.],
       [-1., -1., 13., ...,  0., -1.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0., -1., ...,  0., 11.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  3.]])

In [5]:
np.savetxt('../data/laplace.txt',W,fmt='%0.8f')#用户资源量

In [4]:
#基于初始化的权重梯度下降无中心资源分配算法
def IWDGD(generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,iteration_update,W,iteration):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************IWDGD算法+无计算误差****************************")
    print("相关变量初始值为：")
    for i in range (0,54):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity[i][0])
    print("*************************")
    print("初始资源申请量总和为:",sum_resource[0][0])
    print("*************************")
    print("初始成本总和为:",sum_cost[0][0])
    print("*************************")
    print("迭代步长为:",iteration_update)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,2000):#迭代次数
        iteration.append(k)
 
        #所有用户资源申请量基于可行初始化权重梯度下降的迭代         
        for i in range (0,54):
            weight_gradient_sum = np.float64(0)
            for j in range (0,54):
                weight_gradient_sum=weight_gradient_sum+W[i][j]*(agent_gradient[j][k-1])
                #weight_gradient_sum = np.round(weight_gradient_sum, decimals=3)
            agent_resource_quantity[i][k]=agent_resource_quantity[i][k-1]-iteration_update*weight_gradient_sum
            #agent_resource_quantity[i][k] = np.round(agent_resource_quantity[i][k], decimals=3)       
        
        #计算所有用户的资源量总和
        sum1=np.float64(0)
        for i in range (0,54):
            sum1 = sum1 + agent_resource_quantity[i][k]
        sum_resource[0][k]=sum1
        
        #计算所有用户的成本总和
        sum2=np.float64(0)
        for i in range(0,54):
            sum2=sum2+(generator_c_values[i]*(agent_resource_quantity[i][k])*(agent_resource_quantity[i][k])+generator_b_values[i]*agent_resource_quantity[i][k]+generator_a_values[i])
        sum_cost[0][k]=sum2
        
        #计算当前迭代的用户梯度
        for i in range (0,54):
            agent_gradient[i][k]=2*generator_c_values[i]*agent_resource_quantity[i][k]+generator_b_values[i]
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%50==0:
        #if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,54):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity[i][k])
            print("资源申请量总和为:",sum_resource[0][k])
            print("****************************************************************************")
            print("成本总和为:",sum_cost[0][k])
            print("****************************************************************************")
            print("迭代步长为:",iteration_update)
            print("****************************************************************************")
        
        #iteration_update=1/(pow(k+1,0.05)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
IWDGD(generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,iteration_update,W,iteration)

****************************IWDGD算法+无计算误差****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 111.11111111111111
agent 2 的初始资源申请量为: 111.11111111111111
agent 3 的初始资源申请量为: 111.11111111111111
agent 4 的初始资源申请量为: 111.11111111111111
agent 5 的初始资源申请量为: 111.11111111111111
agent 6 的初始资源申请量为: 111.11111111111111
agent 7 的初始资源申请量为: 111.11111111111111
agent 8 的初始资源申请量为: 111.11111111111111
agent 9 的初始资源申请量为: 111.11111111111111
agent 10 的初始资源申请量为: 111.11111111111111
agent 11 的初始资源申请量为: 111.11111111111111
agent 12 的初始资源申请量为: 111.11111111111111
agent 13 的初始资源申请量为: 111.11111111111111
agent 14 的初始资源申请量为: 111.11111111111111
agent 15 的初始资源申请量为: 111.11111111111111
agent 16 的初始资源申请量为: 111.11111111111111
agent 17 的初始资源申请量为: 111.11111111111111
agent 18 的初始资源申请量为: 111.11111111111111
agent 19 的初始资源申请量为: 111.11111111111111
agent 20 的初始资源申请量为: 111.11111111111111
agent 21 的初始资源申请量为: 111.11111111111111
agent 22 的初始资源申请量为: 111.11111111111111
agent 23 的初始资源申请量为: 111.11111111111111
agent 24 的初始资源申请量为: 111.11111111

49 次迭代中的结果：
agent 1 在 49 次迭代中的资源申请量为: 48.08658430600388
agent 2 在 49 次迭代中的资源申请量为: 60.58323038250485
agent 3 在 49 次迭代中的资源申请量为: 43.52933909353284
agent 4 在 49 次迭代中的资源申请量为: 64.05443314455074
agent 5 在 49 次迭代中的资源申请量为: 64.85443314455077
agent 6 在 49 次迭代中的资源申请量为: 71.57987645900585
agent 7 在 49 次迭代中的资源申请量为: 39.220987610003085
agent 8 在 49 次迭代中的资源申请量为: 37.48993822950293
agent 9 在 49 次迭代中的资源申请量为: 42.82933909353286
agent 10 在 49 次迭代中的资源申请量为: 55.42298189154297
agent 11 在 49 次迭代中的资源申请量为: 45.187422786878656
agent 12 在 49 次迭代中的资源申请量为: 48.98658430600391
agent 13 在 49 次迭代中的资源申请量为: 65.35443314455078
agent 14 在 49 次迭代中的资源申请量为: 54.822981891542966
agent 15 在 49 次迭代中的资源申请量为: 43.22933909353286
agent 16 在 49 次迭代中的资源申请量为: 43.42933909353286
agent 17 在 49 次迭代中的资源申请量为: 44.58742278687866
agent 18 在 49 次迭代中的资源申请量为: 46.18742278687866
agent 19 在 49 次迭代中的资源申请量为: 41.055486921669925
agent 20 在 49 次迭代中的资源申请量为: 55.322981891542966
agent 21 在 49 次迭代中的资源申请量为: 38.52098761000308
agent 22 在 49 次迭代中的资源申请量为: 38.82098761000309
ag

99 次迭代中的结果：
agent 1 在 99 次迭代中的资源申请量为: 48.08658430600388
agent 2 在 99 次迭代中的资源申请量为: 60.58323038250485
agent 3 在 99 次迭代中的资源申请量为: 43.52933909353284
agent 4 在 99 次迭代中的资源申请量为: 64.05443314455074
agent 5 在 99 次迭代中的资源申请量为: 64.85443314455077
agent 6 在 99 次迭代中的资源申请量为: 71.57987645900585
agent 7 在 99 次迭代中的资源申请量为: 39.220987610003085
agent 8 在 99 次迭代中的资源申请量为: 37.48993822950293
agent 9 在 99 次迭代中的资源申请量为: 42.82933909353286
agent 10 在 99 次迭代中的资源申请量为: 55.42298189154297
agent 11 在 99 次迭代中的资源申请量为: 45.187422786878656
agent 12 在 99 次迭代中的资源申请量为: 48.98658430600391
agent 13 在 99 次迭代中的资源申请量为: 65.35443314455078
agent 14 在 99 次迭代中的资源申请量为: 54.822981891542966
agent 15 在 99 次迭代中的资源申请量为: 43.22933909353286
agent 16 在 99 次迭代中的资源申请量为: 43.42933909353286
agent 17 在 99 次迭代中的资源申请量为: 44.58742278687866
agent 18 在 99 次迭代中的资源申请量为: 46.18742278687866
agent 19 在 99 次迭代中的资源申请量为: 41.055486921669925
agent 20 在 99 次迭代中的资源申请量为: 55.322981891542966
agent 21 在 99 次迭代中的资源申请量为: 38.52098761000308
agent 22 在 99 次迭代中的资源申请量为: 38.82098761000309
ag

149 次迭代中的结果：
agent 1 在 149 次迭代中的资源申请量为: 48.08658430600388
agent 2 在 149 次迭代中的资源申请量为: 60.58323038250485
agent 3 在 149 次迭代中的资源申请量为: 43.52933909353284
agent 4 在 149 次迭代中的资源申请量为: 64.05443314455074
agent 5 在 149 次迭代中的资源申请量为: 64.85443314455077
agent 6 在 149 次迭代中的资源申请量为: 71.57987645900585
agent 7 在 149 次迭代中的资源申请量为: 39.220987610003085
agent 8 在 149 次迭代中的资源申请量为: 37.48993822950293
agent 9 在 149 次迭代中的资源申请量为: 42.82933909353286
agent 10 在 149 次迭代中的资源申请量为: 55.42298189154297
agent 11 在 149 次迭代中的资源申请量为: 45.187422786878656
agent 12 在 149 次迭代中的资源申请量为: 48.98658430600391
agent 13 在 149 次迭代中的资源申请量为: 65.35443314455078
agent 14 在 149 次迭代中的资源申请量为: 54.822981891542966
agent 15 在 149 次迭代中的资源申请量为: 43.22933909353286
agent 16 在 149 次迭代中的资源申请量为: 43.42933909353286
agent 17 在 149 次迭代中的资源申请量为: 44.58742278687866
agent 18 在 149 次迭代中的资源申请量为: 46.18742278687866
agent 19 在 149 次迭代中的资源申请量为: 41.055486921669925
agent 20 在 149 次迭代中的资源申请量为: 55.322981891542966
agent 21 在 149 次迭代中的资源申请量为: 38.52098761000308
agent 22 在 149 次迭代中的资源申请量

199 次迭代中的结果：
agent 1 在 199 次迭代中的资源申请量为: 48.08658430600388
agent 2 在 199 次迭代中的资源申请量为: 60.58323038250485
agent 3 在 199 次迭代中的资源申请量为: 43.52933909353284
agent 4 在 199 次迭代中的资源申请量为: 64.05443314455074
agent 5 在 199 次迭代中的资源申请量为: 64.85443314455077
agent 6 在 199 次迭代中的资源申请量为: 71.57987645900585
agent 7 在 199 次迭代中的资源申请量为: 39.220987610003085
agent 8 在 199 次迭代中的资源申请量为: 37.48993822950293
agent 9 在 199 次迭代中的资源申请量为: 42.82933909353286
agent 10 在 199 次迭代中的资源申请量为: 55.42298189154297
agent 11 在 199 次迭代中的资源申请量为: 45.187422786878656
agent 12 在 199 次迭代中的资源申请量为: 48.98658430600391
agent 13 在 199 次迭代中的资源申请量为: 65.35443314455078
agent 14 在 199 次迭代中的资源申请量为: 54.822981891542966
agent 15 在 199 次迭代中的资源申请量为: 43.22933909353286
agent 16 在 199 次迭代中的资源申请量为: 43.42933909353286
agent 17 在 199 次迭代中的资源申请量为: 44.58742278687866
agent 18 在 199 次迭代中的资源申请量为: 46.18742278687866
agent 19 在 199 次迭代中的资源申请量为: 41.055486921669925
agent 20 在 199 次迭代中的资源申请量为: 55.322981891542966
agent 21 在 199 次迭代中的资源申请量为: 38.52098761000308
agent 22 在 199 次迭代中的资源申请量

...

1739 次迭代中的结果：
agent 1 在 1739 次迭代中的资源申请量为: 48.08658430600388
agent 2 在 1739 次迭代中的资源申请量为: 60.58323038250485
agent 3 在 1739 次迭代中的资源申请量为: 43.52933909353284
agent 4 在 1739 次迭代中的资源申请量为: 64.05443314455074
agent 5 在 1739 次迭代中的资源申请量为: 64.85443314455077
agent 6 在 1739 次迭代中的资源申请量为: 71.57987645900585
agent 7 在 1739 次迭代中的资源申请量为: 39.220987610003085
agent 8 在 1739 次迭代中的资源申请量为: 37.48993822950293
agent 9 在 1739 次迭代中的资源申请量为: 42.82933909353286
agent 10 在 1739 次迭代中的资源申请量为: 55.42298189154297
agent 11 在 1739 次迭代中的资源申请量为: 45.187422786878656
agent 12 在 1739 次迭代中的资源申请量为: 48.98658430600391
agent 13 在 1739 次迭代中的资源申请量为: 65.35443314455078
agent 14 在 1739 次迭代中的资源申请量为: 54.822981891542966
agent 15 在 1739 次迭代中的资源申请量为: 43.22933909353286
agent 16 在 1739 次迭代中的资源申请量为: 43.42933909353286
agent 17 在 1739 次迭代中的资源申请量为: 44.58742278687866
agent 18 在 1739 次迭代中的资源申请量为: 46.18742278687866
agent 19 在 1739 次迭代中的资源申请量为: 41.055486921669925
agent 20 在 1739 次迭代中的资源申请量为: 55.322981891542966
agent 21 在 1739 次迭代中的资源申请量为: 38.52098761000308
age

1789 次迭代中的结果：
agent 1 在 1789 次迭代中的资源申请量为: 48.08658430600388
agent 2 在 1789 次迭代中的资源申请量为: 60.58323038250485
agent 3 在 1789 次迭代中的资源申请量为: 43.52933909353284
agent 4 在 1789 次迭代中的资源申请量为: 64.05443314455074
agent 5 在 1789 次迭代中的资源申请量为: 64.85443314455077
agent 6 在 1789 次迭代中的资源申请量为: 71.57987645900585
agent 7 在 1789 次迭代中的资源申请量为: 39.220987610003085
agent 8 在 1789 次迭代中的资源申请量为: 37.48993822950293
agent 9 在 1789 次迭代中的资源申请量为: 42.82933909353286
agent 10 在 1789 次迭代中的资源申请量为: 55.42298189154297
agent 11 在 1789 次迭代中的资源申请量为: 45.187422786878656
agent 12 在 1789 次迭代中的资源申请量为: 48.98658430600391
agent 13 在 1789 次迭代中的资源申请量为: 65.35443314455078
agent 14 在 1789 次迭代中的资源申请量为: 54.822981891542966
agent 15 在 1789 次迭代中的资源申请量为: 43.22933909353286
agent 16 在 1789 次迭代中的资源申请量为: 43.42933909353286
agent 17 在 1789 次迭代中的资源申请量为: 44.58742278687866
agent 18 在 1789 次迭代中的资源申请量为: 46.18742278687866
agent 19 在 1789 次迭代中的资源申请量为: 41.055486921669925
agent 20 在 1789 次迭代中的资源申请量为: 55.322981891542966
agent 21 在 1789 次迭代中的资源申请量为: 38.52098761000308
age

1849 次迭代中的结果：
agent 1 在 1849 次迭代中的资源申请量为: 48.08658430600388
agent 2 在 1849 次迭代中的资源申请量为: 60.58323038250485
agent 3 在 1849 次迭代中的资源申请量为: 43.52933909353284
agent 4 在 1849 次迭代中的资源申请量为: 64.05443314455074
agent 5 在 1849 次迭代中的资源申请量为: 64.85443314455077
agent 6 在 1849 次迭代中的资源申请量为: 71.57987645900585
agent 7 在 1849 次迭代中的资源申请量为: 39.220987610003085
agent 8 在 1849 次迭代中的资源申请量为: 37.48993822950293
agent 9 在 1849 次迭代中的资源申请量为: 42.82933909353286
agent 10 在 1849 次迭代中的资源申请量为: 55.42298189154297
agent 11 在 1849 次迭代中的资源申请量为: 45.187422786878656
agent 12 在 1849 次迭代中的资源申请量为: 48.98658430600391
agent 13 在 1849 次迭代中的资源申请量为: 65.35443314455078
agent 14 在 1849 次迭代中的资源申请量为: 54.822981891542966
agent 15 在 1849 次迭代中的资源申请量为: 43.22933909353286
agent 16 在 1849 次迭代中的资源申请量为: 43.42933909353286
agent 17 在 1849 次迭代中的资源申请量为: 44.58742278687866
agent 18 在 1849 次迭代中的资源申请量为: 46.18742278687866
agent 19 在 1849 次迭代中的资源申请量为: 41.055486921669925
agent 20 在 1849 次迭代中的资源申请量为: 55.322981891542966
agent 21 在 1849 次迭代中的资源申请量为: 38.52098761000308
age

1909 次迭代中的结果：
agent 1 在 1909 次迭代中的资源申请量为: 48.08658430600388
agent 2 在 1909 次迭代中的资源申请量为: 60.58323038250485
agent 3 在 1909 次迭代中的资源申请量为: 43.52933909353284
agent 4 在 1909 次迭代中的资源申请量为: 64.05443314455074
agent 5 在 1909 次迭代中的资源申请量为: 64.85443314455077
agent 6 在 1909 次迭代中的资源申请量为: 71.57987645900585
agent 7 在 1909 次迭代中的资源申请量为: 39.220987610003085
agent 8 在 1909 次迭代中的资源申请量为: 37.48993822950293
agent 9 在 1909 次迭代中的资源申请量为: 42.82933909353286
agent 10 在 1909 次迭代中的资源申请量为: 55.42298189154297
agent 11 在 1909 次迭代中的资源申请量为: 45.187422786878656
agent 12 在 1909 次迭代中的资源申请量为: 48.98658430600391
agent 13 在 1909 次迭代中的资源申请量为: 65.35443314455078
agent 14 在 1909 次迭代中的资源申请量为: 54.822981891542966
agent 15 在 1909 次迭代中的资源申请量为: 43.22933909353286
agent 16 在 1909 次迭代中的资源申请量为: 43.42933909353286
agent 17 在 1909 次迭代中的资源申请量为: 44.58742278687866
agent 18 在 1909 次迭代中的资源申请量为: 46.18742278687866
agent 19 在 1909 次迭代中的资源申请量为: 41.055486921669925
agent 20 在 1909 次迭代中的资源申请量为: 55.322981891542966
agent 21 在 1909 次迭代中的资源申请量为: 38.52098761000308
age

1959 次迭代中的结果：
agent 1 在 1959 次迭代中的资源申请量为: 48.08658430600388
agent 2 在 1959 次迭代中的资源申请量为: 60.58323038250485
agent 3 在 1959 次迭代中的资源申请量为: 43.52933909353284
agent 4 在 1959 次迭代中的资源申请量为: 64.05443314455074
agent 5 在 1959 次迭代中的资源申请量为: 64.85443314455077
agent 6 在 1959 次迭代中的资源申请量为: 71.57987645900585
agent 7 在 1959 次迭代中的资源申请量为: 39.220987610003085
agent 8 在 1959 次迭代中的资源申请量为: 37.48993822950293
agent 9 在 1959 次迭代中的资源申请量为: 42.82933909353286
agent 10 在 1959 次迭代中的资源申请量为: 55.42298189154297
agent 11 在 1959 次迭代中的资源申请量为: 45.187422786878656
agent 12 在 1959 次迭代中的资源申请量为: 48.98658430600391
agent 13 在 1959 次迭代中的资源申请量为: 65.35443314455078
agent 14 在 1959 次迭代中的资源申请量为: 54.822981891542966
agent 15 在 1959 次迭代中的资源申请量为: 43.22933909353286
agent 16 在 1959 次迭代中的资源申请量为: 43.42933909353286
agent 17 在 1959 次迭代中的资源申请量为: 44.58742278687866
agent 18 在 1959 次迭代中的资源申请量为: 46.18742278687866
agent 19 在 1959 次迭代中的资源申请量为: 41.055486921669925
agent 20 在 1959 次迭代中的资源申请量为: 55.322981891542966
agent 21 在 1959 次迭代中的资源申请量为: 38.52098761000308
age

In [6]:
np.savetxt('../data/agent_resource_quantity_oracle.txt',agent_resource_quantity,fmt='%0.8f')#用户资源量
np.savetxt('../data/sum_cost_oracle.txt',sum_cost,fmt='%0.8f')#用户成本总和
np.savetxt('../data/sum_resource_oracle.txt',sum_resource,fmt='%0.8f')#用户资源总和
np.savetxt('../data/agent_gradient_oracle.txt',agent_gradient,fmt='%0.8f')#用户资源总和

In [7]:
agent_gradient[2][0]

np.float64(41.72446666666667)

In [8]:
agent_gradient[32][0]

np.float64(43.986133333333335)

In [9]:
agent_gradient[48][0]

np.float64(43.986133333333335)

In [14]:
D[49]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0.,
       0., 0., 0.])

In [11]:
agent_gradient[0][1998]

np.float64(17.986116937754268)

In [12]:
agent_gradient[24][1998]

np.float64(17.984974110872056)